# Chapter 2 - First Steps with LangChain



## ChatDeepSeek example

Taken from [langchain-deepseek](https://python.langchain.com/api_reference/deepseek/chat_models/langchain_deepseek.chat_models.ChatDeepSeek.html) documentation

In [1]:
import os
from langchain_deepseek import ChatDeepSeek
from dotenv import load_dotenv

load_dotenv() # load environment variables from .env file

# instantiate the llm
llm = ChatDeepSeek(model="deepseek-chat", api_key=os.environ.get("DEEPSEEK_API_KEY"))

# construct a prompt

messages = [
    ("system", "You are a helpful translator. Translate the user sentence to Spanish"),
    ("human", "I love programming.")
]

# pass the prompt to the llm and get a response
response = llm.invoke(messages)
print(response)

content='Me encanta programar.  \n\n*(Alternative options depending on context:)*  \n- **Amo programar.** (stronger emphasis)  \n- **Disfruto mucho programando.** ("I really enjoy programming")  \n\nLet me know if you\'d like a regional variation (e.g., Latin America vs. Spain) or a different tone!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 19, 'total_tokens': 90, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 19}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0623_fp8_kvcache', 'id': 'db3b6cc4-4e7b-4ebf-853c-9b8ee4d94959', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--0fc05e59-ffb8-489c-b02f-323407d02631-0' usage_metadata={'input_tokens': 19, 'output_tokens': 71, 'total_tokens': 90, 'input_token_details': {'cache_read': 0}, 'output_token_details': {

## Development testing

During development, you might want to test your application without making an actual API calls. LangChain provides ```FakeListLLM``` for this purpose:

In [2]:
from langchain_community.llms import FakeListLLM

# create a fake LLM that always returns the same response

fake_llm = FakeListLLM(responses=["Hello There"])

response = fake_llm.invoke("Any input will return 'Hello There'")
print(response)

Hello There


## Working with chat models

In [3]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.messages import SystemMessage, HumanMessage
from dotenv import load_dotenv
import os

load_dotenv() # load environment variables from .env file

# instantiate llm
chat = ChatDeepSeek(model="deepseek-chat", api_key=os.environ.get("DEEPSEEK_API_KEY"))

# construct prompt
messages = [
    SystemMessage(content="You're a helpful programming assistant."),
    HumanMessage(content="Write a Python function to calculate factorial."),
]

# invoke the llm
response = chat.invoke(messages)
print(response)

content='Here’s a Python function to calculate the factorial of a non-negative integer using both iterative and recursive approaches:\n\n### 1. Iterative Approach:\n```python\ndef factorial_iterative(n):\n    """\n    Calculate the factorial of a non-negative integer n using iteration.\n    \n    Args:\n        n (int): Non-negative integer\n    \n    Returns:\n        int: Factorial of n\n    """\n    if n < 0:\n        raise ValueError("Factorial is not defined for negative numbers.")\n    result = 1\n    for i in range(1, n + 1):\n        result *= i\n    return result\n```\n\n### 2. Recursive Approach:\n```python\ndef factorial_recursive(n):\n    """\n    Calculate the factorial of a non-negative integer n using recursion.\n    \n    Args:\n        n (int): Non-negative integer\n    \n    Returns:\n        int: Factorial of n\n    """\n    if n < 0:\n        raise ValueError("Factorial is not defined for negative numbers.")\n    if n == 0 or n == 1:\n        return 1\n    return n 